In [1]:
%matplotlib inline
import sys
sys.path.append('../modeling')
from config import TRACKING_URI
import mlflow

# set the experiment id
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(experiment_name='model_pipeline')
mlflow.autolog()

2025/06/03 22:13:40 INFO mlflow.tracking.fluent: Experiment with name 'model_pipeline' does not exist. Creating a new experiment.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/synth_combined.csv', index_col=0)
col_target = 'dropped_out'
X = df.drop(col_target, axis=1)
# MLFlow doesn't like a series for the target, so convert it to numpy - for sklearn it doesn't matter
y = df[col_target].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

2025/06/03 22:13:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [3]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.pipeline import Pipeline
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

param_default = [{
    'C': [1.0],
    'class_weight': [None],
    'dual': [False],
    'fit_intercept': [True],
    'intercept_scaling': [1],
    'l1_ratio': [None],
    'max_iter': [100],
    'multi_class': ['deprecated'],
    'penalty': ['l2'],
    'random_state': [None],
    'solver': ['lbfgs'],
    'tol': [0.0001],
    'warm_start': [False]
}]

param = [{
    'model_logreg__penalty': ['l2', 'l1'],
    'model_logreg__C': [0.01, 0.1, 1, 10],
    'model_logreg__solver': ['liblinear', 'saga'],
    'model_logreg__max_iter': [200, 500]
}]

pipe = Pipeline([
    ('enc_onehot', OneHotEncoder(drop='first')),
    ('model_logreg', LogisticRegression())
])

model = HalvingGridSearchCV(pipe, param, aggressive_elimination=True, scoring='f1', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

from utils import check_classification_binary
check_classification_binary(model, X_train, X_test, y_train, y_test)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 41718
max_resources_: 1126400
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 32
n_resources: 41718
Fitting 5 folds for each of 32 candidates, totalling 160 fits
----------
iter: 1
n_candidates: 11
n_resources: 125154
Fitting 5 folds for each of 11 candidates, totalling 55 fits
----------
iter: 2
n_candidates: 4
n_resources: 375462
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 1126386
Fitting 5 folds for each of 2 candidates, totalling 10 fits


2025/06/03 22:15:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c41eb318957042aa98c0be5d18e791e8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/06/03 22:15:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/boesi/Projekte/neue-fische/FinApprenticeship/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `

🏃 View run marvelous-roo-211 at: http://mlflow.fa.berufsnavigator.de/#/experiments/783766249587513434/runs/c41eb318957042aa98c0be5d18e791e8
🧪 View experiment at: http://mlflow.fa.berufsnavigator.de/#/experiments/783766249587513434


ImportError: cannot import name 'check_classification_binary' from 'utils' (/home/boesi/Projekte/neue-fische/FinApprenticeship/notebooks/../modeling/utils.py)